In [1]:
from datetime import datetime
from credit_risk.entity import DataIngestionConfig,DataIngestionArtifact
from credit_risk.entity.config_entity import DataValidationConfig
from credit_risk.components.data_validation import DataValidation
from credit_risk.entity.config_entity import  TrainingPipelineConfig
from credit_risk.exception import CreditRiskException
from credit_risk.components.data_ingestion import DataIngestion
import os
from credit_risk.pipeline.training import TrainingPipeline

training_pipeline_config = TrainingPipelineConfig()
tr = TrainingPipeline(training_pipeline_config=training_pipeline_config)
tr.start() 

24/09/16 19:04:40 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 19:04:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 19:04:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Row: [423553] Column: [12]
Expected Column: ['loan_status', 'cb_person_default_on_file', 'person_home_ownership', 'loan_intent', 'loan_grade', 'person_age', 'person_income', 'person_emp_length', 'loan_amnt', 'loan_int_rate', 'loan_percent_income', 'cb_person_cred_hist_length']
Present Columns: ['person_age', 'person_income', 'person_home_ownership', 'person_emp_length', 'loan_intent', 'loan_grade', 'loan_amnt', 'loan_int_rate', 'loan_status', 'loan_percent_income', 'cb_person_default_on_file', 'cb_person_cred_hist_length']


+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+---------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|age_group|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+---------+
|    20-25||       115731|             MORTGAGE|              4.0|DEBTCONSOLIDATION|         B|     9625|        10.74|          0|               0.08|                        N|                        3
|    20-25||       135000|             MORTGAGE|              1.0|          VENTURE|         C|    15000|        15.96|          0|               0.11| 

In [ ]:
from credit_risk.ml.features import AgeGroupCategorizer
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('credit-risk-prediction')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()
data = spark.read.csv("Data/credit_risk_dataset.csv",header=True)
bins = [20, 26, 36, 46, 56, 66, 80]
labels = ['20-25', '26-35', '36-45', '46-55', '56-65', '66-80']

age_group_categorizer = AgeGroupCategorizer(inputCol='person_age', outputCol='age_group', bins=bins, labels=labels)

# Apply the transformer to the training data
training_data_transformed = age_group_categorizer.transform(data)

training_data_transformed.show()

24/09/16 14:13:13 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 14:13:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 14:13:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+---------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|age_group|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+---------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|                        Y|                         3|    20-25|
|        21|         9600|                  OWN|              5.0|        EDUCATION|         B|     1000|        11.14|          0|         

In [ ]:
tf = training_data_transformed.groupBy('age_group').count()
tf.show()


+---------+-----+
|age_group|count|
+---------+-----+
|    26-35|11834|
|    20-25|17829|
|    46-55|  434|
|    36-45| 2360|
|    66-80|   26|
|    56-65|   98|
+---------+-----+



In [ ]:
from credit_risk.ml.features import IncomeGroupCategorizer

income_group_categorizer = IncomeGroupCategorizer(inputCol='person_income', outputCol='income_group')

# Apply the transformer to the training data
training_data_transformed = income_group_categorizer.transform(data)

# Apply the transformer to the validation data if needed
# validation_data_transformed = income_group_categorizer.transform(validation_data)

# Show the transformed data
training_data_transformed.show()

+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+------------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|income_group|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+------------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|                        Y|                         3|      middle|
|        21|         9600|                  OWN|              5.0|        EDUCATION|         B|     1000|        11.14|         

In [ ]:
from credit_risk.ml.features import LoanAmountCategorizer
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('credit-risk-prediction')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()
data = spark.read.csv("Data/credit_risk_dataset.csv",header=True)
loan_amount_categorizer = LoanAmountCategorizer(inputCol='loan_amnt', outputCol='loan_amount_group')

# Apply the transformer to the training data
training_data_transformed = loan_amount_categorizer.transform(data)

# Apply the transformer to the validation data if needed
# validation_data_transformed = income_group_categorizer.transform(validation_data)

# Show the transformed data
training_data_transformed.show()

24/09/16 14:25:32 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 14:25:32 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 14:25:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+-----------------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|loan_amount_group|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+-----------------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|                        Y|                         3|        very high|
|        21|         9600|                  OWN|              5.0|        EDUCATION|         B|     1000|   

In [ ]:
from credit_risk.ml.features import RatioFeatureGenerator
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('credit-risk-prediction')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()
data = spark.read.csv("Data/credit_risk_dataset.csv",header=True)
# Initialize the transformer
ratio_feature_generator = RatioFeatureGenerator()

# Apply the transformer to the training data
training_data_transformed = ratio_feature_generator.transform(data)

training_data_transformed.show()


24/09/16 14:28:26 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 14:28:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 14:28:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+--------------------+------------------------+--------------------------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|loan_to_income_ratio|loan_to_emp_length_ratio|int_rate_to_loan_amt_ratio|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+--------------------+------------------------+--------------------------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|               

In [ ]:
from credit_risk.ml.features import DataCleaner
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('credit-risk-prediction')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()
data = spark.read.csv("Data/credit_risk_dataset.csv",header=True)

data_cleaner = DataCleaner(
    age_column='person_age',
    age_threshold=80,
    column_to_drop='index',
    emp_length_column='person_emp_length',
    emp_length_threshold=60
)


# Cleaning up the validation data
validation_data_transformed = data_cleaner.transform(data)

tt  =validation_data_transformed.toPandas()
tt.isnull().sum()

24/09/16 16:28:24 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 16:28:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 16:28:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


person_age                    0
person_income                 0
person_home_ownership         0
person_emp_length             0
loan_intent                   0
loan_grade                    0
loan_amnt                     0
loan_int_rate                 0
loan_status                   0
loan_percent_income           0
cb_person_default_on_file     0
cb_person_cred_hist_length    0
dtype: int64

In [ ]:
tt.shape

(28631, 12)

In [ ]:
from credit_risk.ml.features import DataUpsampler
from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').appName('credit-risk-prediction')\
.config("spark.executor.instances", "1")\
.config("spark.executor.memory", "6g")\
.config("spark.driver.memory", "6g")\
.config("spark.executor.memoryOverhead", "8g")\
.getOrCreate()
data = spark.read.csv("Data/credit_risk_dataset.csv",header=True)

upsampler = DataUpsampler(data, "loan_status")

upsampled_train_df = upsampler.upsample_data()

# Show class distribution after upsampling
yp = upsampled_train_df.groupBy('loan_status').count()
yp.show()



24/09/16 16:52:26 WARN Utils: Your hostname, langchain resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp3s0)
24/09/16 16:52:26 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/09/16 16:52:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


+-----------+-----+
|loan_status|count|
+-----------+-----+
|          0|25473|
|          1|25473|
+-----------+-----+

